In [1]:
using LinearAlgebra
using RowEchelon
using InvertedIndices

In [2]:
A = [
    2 2 7 -1 ; 
    2 1 2 2 ;
    4 6 1 15 ;
]
rref(A)

3×4 Matrix{Float64}:
 1.0  0.0  0.0   1.0
 0.0  1.0  0.0   2.0
 0.0  0.0  1.0  -1.0

In [3]:
B = [
    2 2 7;
    2 1 2;
    4 6 1;
]
C = [
    -1 2 15
]'
B \ C

3×1 Matrix{Float64}:
  1.0
  2.0
 -1.0

In [4]:
rref([B C])

3×4 Matrix{Float64}:
 1.0  0.0  0.0   1.0
 0.0  1.0  0.0   2.0
 0.0  0.0  1.0  -1.0

In [5]:
# Dependent System
D = [
    3 -4 4;
    1 -1 -2;
    2 -3 6
]

E = [ 7 2 5]'

D \ E

3×1 Matrix{Float64}:
 -2.9999999999999996
 -4.333333333333333
 -0.3333333333333333

In [6]:
rref([D E])

3×4 Matrix{Float64}:
 1.0  0.0  0.0  5.5
 0.0  1.0  0.0  2.75
 0.0  0.0  1.0  0.375

In [7]:
# Test for linear independence: use QR's R matrix
Q,R = qr([D E])

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.801784  -0.154303  -0.57735
 -0.267261  -0.771517   0.57735
 -0.534522   0.617213   0.57735
R factor:
3×4 Matrix{Float64}:
 -3.74166   5.07796  -5.87975      -8.81962
  0.0      -0.46291   4.6291        0.46291
  0.0       0.0       2.22045e-15  -1.77636e-15

In [8]:
minimum(abs.(diag(R)))
# If R has no zero diagonal entries, cols are linearly indep.

2.220446049250313e-15

In [9]:
Q1, R1 = qr(([B C]))

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.408248  -0.267261  -0.872872
 -0.408248  -0.801784   0.436436
 -0.816497   0.534522   0.218218
R factor:
3×4 Matrix{Float64}:
 -4.89898  -6.12372  -4.49073  -12.6557
  0.0       1.87083  -2.93987    6.68153
  0.0       0.0      -5.01901    5.01901

In [10]:
minimum(abs.(diag(R2)))
# Not zero, so linearly indep

UndefVarError: UndefVarError: R2 not defined

In [11]:
a1 = rand(5)
a2 = rand(5)
# Linear Dep:
Z = [a1 a2 a1+a2]
Q2, R2 = qr(Z)
minimum(abs.(diag(R2)))
# R diag min is zero, so linearly dep

2.3055512673781017e-16

In [12]:
function linear_dep(aug_matrix)
    Q, R = qr(aug_matrix)
    return minimum(abs.(diag(R)))
end


linear_dep (generic function with 1 method)

In [13]:
linear_dep(Z)

2.3055512673781017e-16

In [14]:

struct DiagonalRowIndexAndConstantValue
    row::Integer
    value::Float64
end
function aug_solve(aug_matrix::AbstractMatrix, tol=10E-6)
    # deprecated - use LA.jl
    Q, R = qr(aug_matrix)
    lin_dep = minimum(abs.(diag(R)))
    i = 1
    min_constants = 0
    rows = Integer(size(aug_matrix)[1])

    function r_diagonal_value(R::Matrix, index::Integer)::Float64
        return abs(R[index, index])
    end

    diagonal = [DiagonalRowIndexAndConstantValue(row, r_diagonal_value(R, row)) for row in 1:rows]
    sort!(diagonal, by=x->x.value)
    minimums_row = first(diagonal).row
    lin_dep = first(diagonal).value
    r_minimums_constant = abs(R[minimums_row, end])


    if lin_dep < tol
        if r_minimums_constant > tol
            throw("Inconsistent system! No solutions!")
        else
            throw("Dependent system! Infinite solutions!")
        end
    end
    return aug_matrix[:, 1:end-1] \ aug_matrix[:,end]

end

aug_solve (generic function with 2 methods)

In [15]:
aug_solve([B C])

3-element Vector{Float64}:
  1.0
  2.0
 -1.0

In [16]:
# Dep Sys
aug_solve([D E])

String: "Dependent system! Infinite solutions!"

In [17]:
# Inconsistent System
F = [
    1 -1 -2;
    2 -3 6;
    3 -4 4;
]

G = [ 2 5 12]'


aug_solve([F G])




String: "Inconsistent system! No solutions!"

In [18]:
# Determine Type:
# Inconsistent
H = [
    5 12 1;
    2 5 2;
    1 2 -3;
]

I = [10 -1 5]'

aug_solve([H I])

String: "Inconsistent system! No solutions!"

In [19]:
# Det Type:
# Dep
J = [
    5 8 -6;
    3 4 -2;
    1 2 -2;
]

K = [14 8 3]'

aug_solve([J K])

String: "Dependent system! Infinite solutions!"

# Inverses


## Not all matrices have inverses

## Only inverses of square matrices

In [20]:
# inv([a b; c d] = A^-1 = (1/(ad-bc))*[d -b; -c a]

In [21]:
inv([2//1 3//1; -1//1 2//1])

2×2 Matrix{Rational{Int64}}:
 2//7  -3//7
 1//7   2//7

In [22]:
(1/(2*2 - (3 * -1))) * [2 -3; 1 2]

2×2 Matrix{Float64}:
 0.285714  -0.428571
 0.142857   0.285714

In [23]:
M = [3 -1 ; -4 2]
inv(M)

2×2 Matrix{Float64}:
 1.0  0.5
 2.0  1.5

In [24]:
function cofactor(A::AbstractMatrix, T = Float64)
    # deprecated - use LA.jl
    ax = axes(A)
    out = similar(A, T, ax)
    for col in ax[1]
        for row in ax[2]
            out[col, row] = (-1)^(col + row) * det(A[Not(col), Not(row)])
        end
    end
    return out
end

cofactor (generic function with 2 methods)

In [25]:
function adjugate(M::AbstractMatrix)
    return transpose(cofactor(M))
end

adjugate (generic function with 1 method)

In [26]:
(1/det(M))*adjugate(M)

2×2 Matrix{Float64}:
 1.0  0.5
 2.0  1.5

In [27]:
N = [
    2 2 -1;
    0 3 -1;
    -1 -2 1
]

inv(N)

3×3 Matrix{Float64}:
 1.0  -1.11022e-16  1.0
 1.0   1.0          2.0
 3.0   2.0          6.0

In [28]:
inv(Rational.(N))

3×3 Matrix{Rational{Int64}}:
 1//1  0//1  1//1
 1//1  1//1  2//1
 3//1  2//1  6//1

In [29]:
(1/det(N))*adjugate(N)

3×3 Matrix{Float64}:
 1.0  -0.0  1.0
 1.0   1.0  2.0
 3.0   2.0  6.0

# Solving Systems

In [30]:
O = [3 2; -1 1]

P = [7 3]'

Rational.(O) \ P

2×1 Matrix{Rational{Int64}}:
  1//5
 16//5

In [31]:
# Ax = B
# A^-1*A*X = A^-1*B
inv(Rational.(O))*P

2×1 Matrix{Rational{Int64}}:
  1//5
 16//5

In [32]:
Q = [
    2 6 6;
    2 7 6;
    2 7 7
]

R = [
    8;10;9
]

aug_solve([Q R])

3-element Vector{Float64}:
  1.0
  2.0
 -1.0

In [33]:
inv(Q)*R

3-element Vector{Float64}:
  1.0
  2.0
 -1.0

In [34]:
S = [
    1 -1 1;
    0 2 -1;
    2 3 0;
]

T = [8 -7 1]'

aug_solve(Rational.([S T]))

3-element Vector{Rational{Int64}}:
  2//1
 -1//1
  5//1

In [35]:
inv(Rational.(S))*T

3×1 Matrix{Rational{Int64}}:
  2//1
 -1//1
  5//1

In [36]:
U = [
    1 2 5;
    2 3 8;
    -1 1 2
]

V = [2 3 3]'

aug_solve(Rational.([U V]))

3-element Vector{Rational{Int64}}:
 -2//1
 -3//1
  2//1

In [37]:
W = inv(Rational.(U))

3×3 Matrix{Rational{Int64}}:
  2//1  -1//1  -1//1
 12//1  -7//1  -2//1
 -5//1   3//1   1//1

In [38]:
W * V

3×1 Matrix{Rational{Int64}}:
 -2//1
 -3//1
  2//1

# Determinants

In [39]:
X = [
    3 0 0;
    2 1 -5;
    2 5 -1
]

det(X)

72.0

In [40]:
Y = [
    3 1 0;
    -3 4 0;
    -1 3 -5
]

det(Y)

-75.0

In [41]:
Z = [
    1 1 1;
    2 2 2;
    -3 4 -5;
]

det(Z)

0.0

In [42]:
AA = [
    1 2 3;
    2 2 -3;
    3 2 1
]

det(AA)

-20.0

# Cramer's Rule

### X = D_x / D, Y = D_y / D

In [43]:
# 3 Var System:

# X = D_x/D, Y = D_y / D, Z = D_z / D

In [44]:
function cramers(aug_m::AbstractMatrix{T})
    # deprecated - use LA.jl
    M = copy(aug_m[:, 1:end-1])
    xyz = copy(aug_m[:, end])
    D = det(M)
    println("D: $(D)")
    i = 1
    cols = size(M)[2]
    results = Array{T}(undef, cols)
    while i <= cols
        D_ = copy(M)
        D_[:, i] .= xyz
        # println("D_$(i): $(D_)")
        _det = det(D_)
        println("D_$(i): det($(D_)): $(_det)")
        results[i] = _det
        i += 1
    end
    answer = Array{T}(undef, cols)
    for j=1:cols
        answer[j] = results[j]/D
    end
    _xyz = [x for x in answer]
    return hcat(_xyz)
end

TypeError: TypeError: in Type, in parameter, expected Type, got a value of type Adjoint{Int64, Matrix{Int64}}

In [45]:
BB = [
    1 2;
    3 -7;
]

CC = [ 19 -8]'

cramers([BB CC])

MethodError: MethodError: no method matching cramers(::Matrix{Int64})

In [46]:
aug_solve([BB CC])

2-element Vector{Float64}:
 9.000000000000002
 5.000000000000001

In [47]:
DD = [
    2 3;
    5 -7
]

EE = [ 7 -3]'

cramers(Rational.([DD EE]))

MethodError: MethodError: no method matching cramers(::Matrix{Rational{Int64}})

In [48]:
FF = [
    1 1 1;
    2 -1 1;
    -1 3 -1
]

GG = [0 -1 -8]'

cramers([FF GG])

MethodError: MethodError: no method matching cramers(::Matrix{Int64})

In [49]:
aug_solve([FF GG])

3-element Vector{Float64}:
 -5.0
 -2.0
  7.0